20/9/2024
Script per andare a controllare se CUDA è stato installato (check che i driver siano andati a buon fine e tutto sia ok), e per testare le performance di un semplice modello su un dataset con e senza l'uso della gpu.

Per farlo sono andato sul web a scaricare iul driver di cuda e di cudnn.

La mia GPU del pc personale è NVIDIA T1200 Laptop GPU

In [1]:
!nvidia-smi 

Sat Oct 12 12:40:54 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA T1200 Laptop GPU      WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   52C    P8              7W /   40W |     474MiB /   4096MiB |     14%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch

In [3]:
print("CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())

CUDA available: True
Number of GPUs: 1


In [4]:
# Crea un tensore
x = torch.randn(3, 3)

# Sposta il tensore sulla GPU
if torch.cuda.is_available():
    x = x.to('cuda')

print(x)

tensor([[ 2.0299, -1.4311, -0.1700],
        [ 1.0307,  0.7661,  0.5978],
        [ 0.8392, -0.2288, -0.5565]], device='cuda:0')


In [5]:
import polars as pl
import pandas as pd

In [10]:
iris = pd.read_csv("Iris.csv")

In [11]:
for it in range (10):
    iris = pd.concat([iris,iris])

In [12]:
iris

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [13]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Carica il DataFrame
df = iris

# Prepara i dati
X = df.drop(columns=['Id', 'Species']).values
y = df['Species'].values

# Codifica le etichette
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Suddividi in train e test
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Converti in tensori
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [14]:
import torch.nn as nn
import torch.optim as optim

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 3)  # 3 classi di specie

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [15]:
def train_model(model, X_train, y_train, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    for epoch in range(100):  # 100 epoche
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train.to(device))
        loss = criterion(outputs, y_train.to(device))
        loss.backward()
        optimizer.step()

In [24]:
%%timeit -n 3 -r 3

device_gpu = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_gpu = SimpleNN()
train_model(model_gpu, X_train_tensor, y_train_tensor, device_gpu)

1.56 s ± 5.38 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [25]:
%%timeit -n 3 -r 3

device_cpu = torch.device("cpu")
model_cpu = SimpleNN()
train_model(model_cpu, X_train_tensor, y_train_tensor, device_cpu)

10.4 s ± 577 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


Notiamo come i tempi di training dei due modelli siano molto diversi (circa 10 volte tanto senza gpu).
Risultati ottenuti con X_train_tensor di 122800 righe.

In [23]:
def evaluate_model(model, X_test, y_test, device):
    model.to(device)
    model.eval()
    with torch.no_grad():
        outputs = model(X_test.to(device))
        _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted.cpu() == y_test).sum().item() / len(y_test)
    return accuracy

accuracy_cpu = evaluate_model(model_cpu, X_test_tensor, y_test_tensor, device_cpu)
accuracy_gpu = evaluate_model(model_gpu, X_test_tensor, y_test_tensor, device_gpu)

print(f'Accuracy (CPU): {accuracy_cpu:.2f}')
print(f'Accuracy (GPU): {accuracy_gpu:.2f}')

Accuracy (CPU): 0.98
Accuracy (GPU): 0.99


In [ ]:
predi